In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import shutil
import tqdm

In [ ]:
raw_sequence_name = [
    '2011_10_03_drive_0027', '2011_10_03_drive_0042',
    '2011_10_03_drive_0034', None,
    '2011_09_30_drive_0016', '2011_09_30_drive_0018',
    '2011_09_30_drive_0020', '2011_09_30_drive_0027',
    '2011_09_30_drive_0028', '2011_09_30_drive_0033',
    '2011_09_30_drive_0034'
]
raw_sequence_start = [0] * 8 + [1100] + [0] * 2
raw_sequence_end = [
    4540, 1100, 4660, 800, 270, 2760, 1100, 1100, 5170, 1590, 1200, 382]

In [ ]:
kitti_root = Path('/data/datasets/kitti/')
kitti_unsynced = kitti_root / "raw_unsynced"
kitti_synced = kitti_root / "raw_synced"
kitti_fixed = kitti_root / "raw_fixed"
kitti_odom = kitti_root / "data_odometry_gray"

In [ ]:
# Odometry Seq
SEQUENCE = 10
kitti_subset = raw_sequence_name[SEQUENCE]
kitti_set = kitti_subset[:10]

In [ ]:
fixed_oxts_file = kitti_fixed / kitti_set / kitti_subset / 'oxts-fixed' / "data.txt"
fixed_oxts = pd.read_csv(fixed_oxts_file, sep=',', header=None, index_col=False)
# add column names
fixed_oxts.columns = ['timestamp', 'lat', 'lon', 'alt', 'roll', 'pitch', 'yaw', 'vn', 've', 'vf', 'vl', 'vu', 'ax', 'ay', 'az', 'af',
                      'al', 'au', 'wx', 'wy', 'wz', 'wf', 'wl', 'wu', 'pos_accuracy', 'vel_accuracy', 'navstat',
                      'numsats', 'posmode', 'velmode', 'orimode']


# convert timestamp to nanoseconds
fixed_oxts['timestamp'] = fixed_oxts['timestamp'] * 1e9


# to ORB-SLAM2 format

# select timestamp, wx, wy, wz, ax, ay, az
fixed_oxts = fixed_oxts[['timestamp', 'wx', 'wy', 'wz', 'ax', 'ay', 'az']]

# format timestamp to string without decimals
fixed_oxts['timestamp'] = fixed_oxts['timestamp'].apply(lambda x: f'{x:.0f}')
# print length
print(fixed_oxts.shape)
# remove NaNs
fixed_oxts = fixed_oxts.dropna()
# print length
print(fixed_oxts.shape)
# write to file
output_file = kitti_fixed / "KITTI_IMU" / f'{SEQUENCE:02}.txt'
output_file.parent.mkdir(parents=True, exist_ok=True)
with open(output_file, 'w') as file:
    file.write("#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]\n")
    fixed_oxts.to_csv(file, sep=',', header=False, index=False)


In [ ]:
# Read image timestamps
image_timestamps_file = kitti_synced / kitti_set / f"{kitti_subset}_sync" / 'image_00' / "timestamps.txt"
# def read_timestamp_file(file):
#     with open(file, 'r') as file:
#         data = file.readlines()
#     return [pd.to_datetime(ts, format="%Y-%m-%d %H:%M:%S.%f\n") 
#         for ts in data]

# def datetimes_to_unix_timestamps(datetimes):
#     return np.array([
#         (t - np.datetime64('1970-01-01T00:00:00')).astype(int)
#             for t in pd.to_datetime(datetimes).to_numpy()])

# # read timestamps
# image_timestamps = read_timestamp_file(image_timestamps_file)
# # convert to unix timestamps
# image_timestamps = datetimes_to_unix_timestamps(image_timestamps)  # in nanoseconds
# print(image_timestamps.shape)

image_timestamps = pd.read_csv(image_timestamps_file, sep=' ', header=None, index_col=False)

index_start = raw_sequence_start[SEQUENCE]
index_end = raw_sequence_end[SEQUENCE] + 1
# select timestamps
image_timestamps = image_timestamps[index_start:index_end]

# write timestamps in nanoseconds without decimals
output_file = kitti_fixed / "KITTI_TimeStamps" / f'{SEQUENCE:02}.txt'
output_file.parent.mkdir(parents=True, exist_ok=True)
with open(output_file, 'w') as file:
    for timestamp in image_timestamps:
        file.write(f'{timestamp:.0f}\n')

# Copy images
# image_source = kitti_synced / kitti_set / f"{kitti_subset}_sync" / 'image_00' / "data"
image_source = kitti_odom / "sequences" / f"{SEQUENCE:02}" / "image_0"
euroc_like_dest = kitti_fixed / "KITTI_Images" / f'{SEQUENCE:02}' / "mav0"
image_dest = euroc_like_dest / "cam0" / "data"
image_dest.mkdir(parents=True, exist_ok=True)

for index, timestamp in tqdm.tqdm(zip(range(index_start, index_end), image_timestamps)):
    shutil.copyfile(
        # image_source / f'{index:010}.png',
        image_source / f'{index:06}.png',
        image_dest / f'{timestamp:.0f}.png')

# Copy IMU
target_file = euroc_like_dest / "imu0" / "data.csv"
target_file.parent.mkdir(parents=True, exist_ok=True)
shutil.copyfile(
    kitti_fixed / "KITTI_IMU" / f'{SEQUENCE:02}.txt',
    target_file
)